In [1]:
%matplotlib inline

In [2]:
from torchsig.transforms.dataset_transforms import Spectrogram
from torchsig.transforms.target_transforms import YOLOLabel
from torchsig.signals.signal_lists import TorchSigSignalLists
from tqdm.notebook import tqdm

root        = "./datasets/real"

fft_size               = 1024
num_signals_max        = 2
impairment_level       = 1        # 0, 1, 2, 3(?)
snr_db_min             = 0.0      # 0dB - 50dB
snr_db_max             = 50.0
signal_duration_min    = 0.002    # 0.6e-07(?) - 0.010(?)
signal_duration_max    = 0.0025
signal_bandwidth_min   = 10e6
signal_bandwidth_max   = 12.5e6
signal_center_freq_min = -25e6    # -50MHz - 50MHz
signal_center_freq_max = 25e6
num_iq_samples_dataset = fft_size ** 2

class_list  = ["64ask","fm","ofdm-2048","16fsk","chirpss"]
num_classes = len(class_list)

num_train   = 250
num_val     = 50

transforms = [Spectrogram(fft_size=fft_size)]
target_transforms = [YOLOLabel()]

In [3]:
# WIDEBAND DATASET
from torchsig.datasets.dataset_metadata import WidebandMetadata
from torchsig.datasets.datamodules import WidebandDataModule
from torchsig.datasets.wideband import NewWideband

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# DATASET PARAMETERS
dataset_metadata = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size               = fft_size,
    impairment_level       = impairment_level,
    num_signals_max        = num_signals_max,
    snr_db_min             = snr_db_min,
    snr_db_max             = snr_db_max,
    signal_duration_min    = signal_duration_min,
    signal_duration_max    = signal_duration_max,
    signal_bandwidth_min   = signal_bandwidth_min,
    signal_bandwidth_max   = signal_bandwidth_max,
    signal_center_freq_min = signal_center_freq_min,
    signal_center_freq_max = signal_center_freq_max,
    transforms             = transforms,
    class_list             = class_list,
    target_transforms      = target_transforms,
)
wideband = NewWideband( dataset_metadata = dataset_metadata )

0it [00:00, ?it/s]/home/ic3/.conda/envs/sigclass/lib/python3.11/site-packages/torchsig/datasets/datasets.py:338: FutureWarning: NewTorchSigDataset will become a torch.IterableDataset in the future.
  warnings.warn("NewTorchSigDataset will become a torch.IterableDataset in the future.",


In [ ]:
import cv2
import shutil
import os

def prepare_dataset(dataset, train: bool, root: str, start_index: int, stop_index: int):
    os.makedirs(root, exist_ok = True)
    train_path = "train" if train else "val"
    label_dir = f"{root}/labels/{train_path}"
    image_dir = f"{root}/images/{train_path}"
    os.makedirs(label_dir, exist_ok = True)
    os.makedirs(image_dir, exist_ok = True)

    for i in tqdm(range(start_index, stop_index), desc=f"Writing {train_path.title()}"):
        image, labels = dataset[i]
        filename_base = str(i).zfill(10)
        label_filename = f"{label_dir}/{filename_base}.txt"
        image_filename = f"{image_dir}/{filename_base}.png"

        with open(label_filename, "w") as f:
            line = ""
            f.write("\n".join(f"{x[0]} {x[1]} {x[2]} {x[3]} {x[4]}" for x in labels))
        cv2.imwrite(image_filename, image, [cv2.IMWRITE_PNG_COMPRESSION, 9])

if os.path.exists(root):
    shutil.rmtree(root)

prepare_dataset(wideband, train=True, root=root, start_index=1, stop_index = num_train)
prepare_dataset(wideband, train=False, root=root, start_index=num_train, stop_index = num_train + num_val)


Writing Train:   0%|          | 0/249 [00:00<?, ?it/s]

[ WARN:0@0.181] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


In [30]:
import yaml
import torch

config_name = "nosleep.yaml"
classes = {v: k for v,k in enumerate(class_list)}

yolo_config = dict(
    path = "nosleep",
    train = "images/train",
    val = "images/val",
    nc = num_classes,
    names = classes
)

with open(config_name, 'w+') as file:
    yaml.dump(yolo_config, file, default_flow_style=False)

In [18]:
from ultralytics import YOLO
import torch
import requests
import os

config_name = "nosleep.yaml"
model_filepath = "yolov8x.pt"
model = YOLO(model_filepath)
torch.cuda.empty_cache()

In [19]:

%%capture output
%matplotlib inline
results = model.train(
    data=config_name, 
    epochs=50,
    patience=10,
    batch=16,
    imgsz=fft_size,
    device="cuda",
    workers=1,
    project="nosleep",
    name="nosleep",
    verbose=False
)

New https://pypi.org/project/ultralytics/8.3.169 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.160 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=nosleep.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=nosleep7, nbs=64, nms=False, opset=None, optimize=Fals

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 2.56 MiB is free. Including non-PyTorch memory, this process has 23.62 GiB memory in use. Of the allocated memory 23.09 GiB is allocated by PyTorch, and 57.54 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# EVAL
import matplotlib.pyplot as plt
import cv2
import os

results_img = cv2.imread(os.path.join(results.save_dir, "results.png"))
plt.figure(figsize = (10,20))
plt.imshow(results_img)
label = cv2.imread(os.path.join(results.save_dir, "val_batch1_labels.jpg"))
pred = cv2.imread(os.path.join(results.save_dir, "val_batch1_pred.jpg"))
f, ax = plt.subplots(1, 2, figsize=(15, 9))
ax[0].imshow(label)
ax[0].set_title("Label")
ax[1].imshow(pred)
ax[1].set_title("Prediction")
plt.show()

In [20]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 27           |        cudaMalloc retries: 27        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  23642 MiB |  23642 MiB |  37625 MiB |  13983 MiB |
|       from large pool |  23557 MiB |  23557 MiB |  37385 MiB |  13827 MiB |
|       from small pool |     84 MiB |     84 MiB |    240 MiB |    155 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  23642 MiB |  23642 MiB |  37625 MiB |  13983 MiB |
|       from large pool |  23557 MiB |  23557 MiB |  37385 MiB |